In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import SMOTE
plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

In [2]:
df = pd.read_csv('C:\spring2025\SMAI\Project\output.csv')

In [3]:
import itertools
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb

# Base features
base_features = ['inningNumber', 'oversActual', 'totalRuns', 'totalWickets',
                 'time_of_day', 'Ground Name', 'Batsman_Name', 'Batsman_Role',
                 'Batsman_Batting_Style', 'Batsman_Playing_Role']

remove_candidates = ['time_of_day', 'oversActual', 'Batsman_Batting_Style']
target_outputs = ['pitchLine', 'pitchLength', 'Bowler_Bowling_Style', 'shotType']

threshold = 2
df_resampled=df
df_copy = df_resampled.copy()
df_copy['target'] = ((df_copy['isWicket'] == 1) | (df_copy['run'] <= threshold)).astype(int)
df_good = df_copy[df_copy['target'] == 1]

print("\n====== XGBoost Ablation Study ======")

for target in target_outputs:
    y = df_good[target]
    class_counts = y.value_counts()
    valid_classes = class_counts[class_counts > 1].index
    dropped_classes = class_counts[class_counts <= 1]

    if not dropped_classes.empty:
        print(f"\nTarget '{target}': Dropping {len(dropped_classes)} class(es): {list(dropped_classes.index)}")

    mask = y.isin(valid_classes)
    y_filtered = y[mask]

    if y_filtered.nunique() < 2:
        print(f"Skipping target '{target}' due to insufficient class diversity.")
        continue

    for r in range(len(remove_candidates) + 1):
        for remove_subset in itertools.combinations(remove_candidates, r):
            removed = list(remove_subset)
            remaining_features = [f for f in base_features if f not in removed]

            print(f"\n--- Target: {target} | Removing: {', '.join(removed) if removed else 'None'} ---")

            X_all = pd.get_dummies(df_good[remaining_features])
            X_filtered = X_all[mask]

            X_train, X_test, y_train, y_test = train_test_split(
                X_filtered, y_filtered, test_size=0.2, random_state=42, stratify=y_filtered
            )

            model = xgb.XGBClassifier(
                objective='multi:softmax',
                eval_metric='mlogloss',
                use_label_encoder=False,
                random_state=42
            )

            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Accuracy: {accuracy:.4f}")



====== XGBoost Ablation Study ======

--- Target: pitchLine | Removing: None ---


C:\Users\eswar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [07:10:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.6241

--- Target: pitchLine | Removing: time_of_day ---


C:\Users\eswar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [07:10:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.6222

--- Target: pitchLine | Removing: oversActual ---


...

Accuracy: 0.2851

--- Target: shotType | Removing: oversActual, Batsman_Batting_Style ---


C:\Users\eswar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [07:12:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.2894

--- Target: shotType | Removing: time_of_day, oversActual, Batsman_Batting_Style ---


C:\Users\eswar\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [07:13:00] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.2782
